In [ ]:
import cv2
import pandas as pd

import modules.model as model
import modules.changeDetector as changeDetector

model_flow = model.stockChecker()


In [23]:
def annotate(img,bbox):
    lw = 5
    for (x,y,w,h),label,state in bbox:
        if state == "zero":
            img = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), lw) #빨강
        if state == 'new': 
            img = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), lw) #파랑
        if state == 'sub':
            img = cv2.rectangle(img, (x, y), (x+w, y+h), (147, 20, 255), lw) # 자주
        if state == 'add':
            img = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 191, 0), lw) # 하늘

    return img

In [29]:
cap = cv2.VideoCapture("/opt/ml/project/final-project-level3-cv-12/dataset/TESTVIDEO/Test4.mp4")
bbox = []
ramen = {}
first = True
human = False
cnt = 0
flag = False
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter("./output_test_stream.mp4",fourcc,cap.get(cv2.CAP_PROP_FPS),(1440,1440))
for cnt in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    if cnt % 5 != 0:
        ret ,image = cap.read()
        if not ret:
            break
        if bbox:
            image = annotate(image,bbox)
        out.write(image)
        continue
    ret , image = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    if first:
        output = model_flow.ramen_detect(image)
        for i in output:
            check_label = model_flow.ramen_class(image,[i])
            if check_label[0] in ramen.keys():
                ramen[check_label[0]]["base"] += 1
            else:
                ramen[check_label[0]] = {"base" : 1 , "diff" : 0}
        check_image = image.copy()
        first = False
        ramen = pd.DataFrame(ramen).transpose()
        # ramen.to_csv('/opt/ml/project/final-project-level3-cv-12/front/stock.csv')
        
        
    if not human:
        human = model_flow.check_human(image)  # 
    if human and not model_flow.check_human(image):
        _, next_image = cap.read()
        if changeDetector.get_diff(image,next_image).mean() < 0.95:
            flag = True
        else:
            bbox= model_flow.check(check_image,image,is_topDown=True)
            check_image = image.copy()
            if bbox:
                for (x,y,w,h) , label,state in bbox:
                    if state in ['sub','zero']:
                        ramen.loc[label,'diff'] -= 1
                    else:
                        ramen.loc[label,'diff'] += 1
                        
                image = annotate(image,bbox)
                ramen.to_csv('/opt/ml/project/final-project-level3-cv-12/front/stock.csv')
            human = False
            
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    out.write(image)
    if flag:
        out.write(next_image)
        flag = False
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
load